In [18]:
import pickle

pickle_file = '-images.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    X_train = save['X_train']
    y_train = save['y_train']
    X_test = save['X_test']
    y_test = save['y_test']
    del save  # hint to help gc free up memory
    print('Training set', X_train.shape, y_train.shape)
    print('Test set', X_test.shape, y_test.shape)

('Training set', (1470, 50, 50, 3), (1470, 1))
('Test set', (630, 50, 50, 3), (630, 1))


In [19]:
## implement your CNN starting here.
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from keras import backend as K

In [20]:
# number of classes
num_classes = 2 #reducing number of classes 

# image dimensions
img_rows, img_cols = X_train.shape[1],  X_train.shape[2]

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3) #changing from 1 to 3 for third dimension 

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

print X_train.shape
print y_train.shape

print input_shape

(1470, 50, 50, 3)
(1470, 1)
(50, 50, 3)


In [21]:
# model hyperparameters
batch_size = 128 # *increasing the number of training examples in one forward/backward pass--> making it too large can
# cause the kernel to keep dying 
nb_epoch = 30

# network architecture
patch_size_1 = 4 #reducing the patch size for deeper neural network/more extensive connections
patch_size_2 = 4
patch_size_3 = 4 

depth_1 = 40
depth_2 = 50 #increasing depth size, but not too much to the hundreds 
depth_3 = 50

pool_size = 2

num_hidden_1 = 100
num_hidden_2 = 100

dropout = 0.3

In [22]:
# create new Keras Sequential model
model = Sequential()

# add first convolutional layer to model and specify it's depth and filter size
# for the first layer we also have to specify the size of each input image
# which we calculated above
model.add(Convolution2D(depth_1, patch_size_1, patch_size_1,
                        border_mode='valid',
                        input_shape=input_shape))
# apply 'relu' activation function for first layer
model.add(Activation('relu'))
# apply max pooling to reduce the size of the image by a factor of 2
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

# repeat these operations for the second convolutional layer
# this time Keras can figure out the input size 
# from the previous layer on it's own
model.add(Convolution2D(depth_2, patch_size_2, patch_size_2,
                        border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(depth_3, patch_size_3, patch_size_3,
                        border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


# flatten the three-dimensional convolutional layer to a single layer of neurons
model.add(Flatten())

# add the first fully connected layer, applying 'relu' activation and dropout
model.add(Dense(num_hidden_1))
model.add(Activation('relu'))
model.add(Dropout(dropout))

# add the second fully connected layer
model.add(Dense(num_hidden_2))
model.add(Activation('relu'))
model.add(Dropout(dropout))

# add the final classification layer with the number of neurons 
# matching the number of classes we are trying to learn
model.add(Dense(num_classes))

# apply the 'softmax' activation to the final layer to convert the output to 
# a probability distribution
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [24]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 1470 samples, validate on 630 samples
Epoch 1/30
1470/1470 [==============================] - 12s - loss: 0.6619 - acc: 0.9197 - val_loss: 0.4794 - val_acc: 0.9286
Epoch 2/30
1470/1470 [==============================] - 13s - loss: 0.3026 - acc: 0.9279 - val_loss: 0.2595 - val_acc: 0.9286
Epoch 3/30
1470/1470 [==============================] - 11s - loss: 0.2632 - acc: 0.9279 - val_loss: 0.2358 - val_acc: 0.9286
Epoch 4/30
1470/1470 [==============================] - 12s - loss: 0.2458 - acc: 0.9279 - val_loss: 0.2052 - val_acc: 0.9286
Epoch 5/30
1470/1470 [==============================] - 11s - loss: 0.2281 - acc: 0.9279 - val_loss: 0.1355 - val_acc: 0.9286
Epoch 6/30
1470/1470 [==============================] - 11s - loss: 0.1778 - acc: 0.9279 - val_loss: 0.0935 - val_acc: 0.9286
Epoch 7/30
1470/1470 [==============================] - 11s - loss: 0.1716 - acc: 0.9279 - val_loss: 0.0970 - val_acc: 0.9286
Epoch 8/30
1470/1470 [==============================] - 11s - loss: 0.1

In [25]:
#visualization though? 

In [26]:
score = model.evaluate(X_test, Y_test, verbose=0)

print 'Test score:', score[0]
print 'Test accuracy: {:.2%}'.format(score[1])

Test score: 0.0476633516857
Test accuracy: 97.94%


In [30]:
checkpoint_name = "-modelCairo.hdf5"
checkpointer = ModelCheckpoint(checkpoint_name, verbose=0, save_best_only=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])